In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from urllib.parse import urlparse, parse_qs

import numpy as np
import pandas as pd
import re
from urllib.parse import unquote



In [2]:
import undetected_chromedriver as uc

options = uc.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--start-maximized")
# options.add_argument("--disable-extensions")
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--ignore-certificate-errors")
# options.add_argument("--ignore-ssl-errors")

driver = uc.Chrome(options=options)

In [ ]:
def get_hrefs_from_page(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "js__product-link-for-product-id")))
    elements = driver.find_elements(By.CLASS_NAME, "js__product-link-for-product-id")
    return [element.get_attribute("href") for element in elements]

base_url = "https://batdongsan.com.vn/nha-dat-ban-hung-yen"
all_hrefs = []
page = 1

while True:
    url = f"{base_url}/p{page}" if page > 1 else base_url
    print(f"Đang xử lý trang {page}: {url}")
    
    hrefs = get_hrefs_from_page(url)
    all_hrefs.extend(hrefs)
    
    next_page = driver.find_elements(By.CSS_SELECTOR, 'a.re__pagination-icon:not(.re__pagination-icon--no-effect)')
    if not next_page:
        break
    
    page += 1
    time.sleep(3)  

print(f"Tổng số href đã lấy được: {len(all_hrefs)}")
for href in all_hrefs:
    print(href)

driver.quit()

In [7]:
with open("hrefs.txt", "w", encoding="utf-8") as file:
    for href in all_hrefs:
        file.write(href + "\n")

print("Đã lưu tất cả href vào file hrefs.txt")

Đã lưu tất cả href vào file hrefs.txt


In [7]:
def extract_property_info(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "re__pr-specs-content-item-value")))
    
    info = {
        'Link': url,
        'Tiêu đề': np.nan,
        'Địa chỉ': np.nan,
        'Mức giá': np.nan,
        'Giá/m²': np.nan,
        'Số phòng ngủ': np.nan,
        'Huyện': np.nan,
        'Diện tích': np.nan,
        'Mặt tiền': np.nan,
        'Đường vào': np.nan,
        'Hướng nhà': np.nan,
        'Hướng ban công': np.nan,
        'Số tầng': np.nan,
        'Số toilet': np.nan,
        'Pháp lý': np.nan,
        "Latitude": np.nan,
        "Longitude": np.nan,
    }
    
    try:
        title_element = driver.find_element(By.CLASS_NAME, "re__pr-title")
        info['Tiêu đề'] = title_element.text.strip()
    except:
        pass
    
    try:
        address_element = driver.find_element(By.CLASS_NAME, "re__pr-short-description")
        info['Địa chỉ'] = address_element.text.strip()
    except:
        pass
    
    try:
        price_element = driver.find_element(By.CSS_SELECTOR, ".re__pr-short-info-item .value")
        info['Mức giá'] = price_element.text.strip()
        
        price_per_m2_element = driver.find_element(By.CSS_SELECTOR, ".re__pr-short-info-item .ext")
        info['Giá/m²'] = price_per_m2_element.text.strip()
    except:
        pass
    
    try:
        rooms_element = driver.find_element(By.XPATH, "//div[contains(@class, 're__pr-short-info-item') and .//span[contains(text(), 'Phòng ngủ')]]//span[@class='value']")
        info['Số phòng ngủ'] = rooms_element.text.strip()
    except:
        pass
    
    try:
        breadcrumbs = driver.find_elements(By.CSS_SELECTOR, ".re__breadcrumb .re__link-se")
        huyen = next((b.text.strip() for b in breadcrumbs if b.get_attribute("level") == "3"), np.nan)
        info['Huyện'] = huyen
    except:
        pass

    try:
        iframe_element = driver.find_element(By.CSS_SELECTOR, "iframe.lazyload")
        data_src = iframe_element.get_attribute("data-src")
        coords = data_src.split("q=")[1].split(",")
        latitude = coords[0].strip()
        longitude = coords[1].split("&")[0].strip()
        info['Latitude'] = latitude
        info['Longitude'] = longitude

    except:
        pass

    specs = driver.find_elements(By.CLASS_NAME, "re__pr-specs-content-item")
    for spec in specs:
        try:
            title = spec.find_element(By.CLASS_NAME, "re__pr-specs-content-item-title").text.strip()
            value = spec.find_element(By.CLASS_NAME, "re__pr-specs-content-item-value").text.strip()
            if title in info:
                info[title] = value
        except:
            pass
    
    return info

In [ ]:
def create_driver():
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.page_load_strategy = 'normal'
    return uc.Chrome(options=options)

driver = create_driver()
# Đọc danh sách URLs từ file
with open("hi.txt", "r", encoding="utf-8") as file:
    urls = file.read().splitlines()


csv_filename = "data_bds.csv"
total_urls = len(urls)

for index, url in enumerate(urls, start=1):
    try:
        print(f"Đang xử lý link thứ {index}/{total_urls}: {url}")
        property_info = extract_property_info(url)
        
        df = pd.DataFrame([property_info])
        if index == 1:
            df.to_csv(csv_filename, mode='w', header=True, index=False, encoding='utf-8-sig')
        else:
            df.to_csv(csv_filename, mode='a', header=False, index=False, encoding='utf-8-sig')
        
        print(f"Đã lưu thông tin của link thứ {index} vào file CSV")
        time.sleep(1)  
    except Exception as e:
        print(f"Lỗi khi xử lý link thứ {index}/{total_urls} - {url}: {str(e)}")

        driver.quit()
        driver = create_driver()

driver.quit()
